![CoSAppLogo](images/cosapp.svg) **CoSApp** tutorials:

# Drivers

## What is a `Driver`?!

`Driver` objects allow users to modify the state of a `System` for a given purpose. Resolution of non-linear equations, optimization, or time integration of dynamic equations are typical examples of `Driver` usage.
As will be seen in this tutorial, drivers can also be nested, to create a simulation workflow.

## Introduction

### Add a `Driver`

Simply use the `add_driver` method passing the `Driver` object you want to use 
(see [Available drivers](#Available-Drivers) section of this tutorial).

In [ ]:
from cosapp.tests.library.systems import Multiply1
from cosapp.drivers import RunOnce

m = Multiply1('mult')
run = m.add_driver(RunOnce('run', verbose=True))

`Multiply1` *is the same* `System` *as* `Multiply` *in the* [Systems](01-Systems.ipynb) *tutorial*

 ![Driver in system](images/drivers_1.svg)

### Implementation

Every `System` (including sub-systems) may have one or multiple `Driver` objects. They are stored in the `drivers` attribute.
By default, there is no `Driver` attached to a `System`.

The `run_drivers` method of `System` executes drivers recursively. In the following example, the simplest driver
[RunOnce](#RunOnce) is added and executed.

In [ ]:
from cosapp.tests.library.systems import Multiply1

m = Multiply1('mult')
m.add_driver(RunOnce('run'))
print(m.drivers) # print drivers of the system

m.p_in.x = 15.
m.K1 = 2.
m.run_drivers()
print(m.p_out.x)

### Subdrivers - Similarities with `System`

Like a `System`, a `Driver` may have children which also inherit from the base class `Driver`. This allows users to define more complex simulations such as workflows, multi-point design, design of experiments, optimization, *etc.*

By construction, a `System` can have as many levels of drivers as required. These drivers are stored in the `children` attribute of class `Driver`. Nested drivers are created with method `add_child` of class `Driver`.


![drivers](images/drivers_2.svg)


In [ ]:
m = Multiply1('mult')
run = m.add_driver(RunOnce('run', verbose=True))

print(run.children) # the 'run' driver has no child

subrun = run.add_child(RunOnce('subrun')) # add a child `Driver` called 'subrun'
print(run.children, subrun.children)

## Available Drivers

**CoSApp** comes with a set of drivers to help users build their simulations.

### RunOnce

As the name suggests, `RunOnce` makes your `System` and its subsystems compute their code once. It does not deal with residues or iterative loops that may be necessary to resolve the `System`. Instead, it merely transports information from the top system down to the lowest level sub-systems.

To illustrate this point, we define below a new `System` containing an equation, and execute a `RunOnce` driver.

In [ ]:
from cosapp.systems import System
from cosapp.tests.library.ports import XPort

class MultiplyWithResidue(System):

    def setup(self):
        self.add_input(XPort, 'p_in', {'x': 1.})
        self.add_inward('K1', 5.)
        # expected_output will be used as a target for p_out.x variable
        self.add_inward('expected_output', 7.5)
        self.add_output(XPort, 'p_out', {'x': 1.})
        
        # create a new equation: expected_output = p_out.x
        self.add_equation('expected_output == p_out.x') 

    def compute(self):
        self.p_out.x = self.p_in.x * self.K1


In [ ]:
from cosapp.drivers import RunOnce

m = MultiplyWithResidue('mult')
run = m.add_driver(RunOnce('run', verbose=True))

m.run_drivers()

print('List of defined drivers\n', m.drivers)
print('\np_in.x\tK1\tp_out.x\tresidue')
print(m.p_in.x, m.K1, m.p_out.x, m.residues.values(), sep="\t")

### NonLinearSolver

This `Driver` determines the parameters of your `System` declared as unknowns that satisfy its equations. It resolves the mathematical problem between free parameters and residues of its child drivers.

`NonLinearSolver` is primarily designed to solve [multi-point problems](./08-Multipoints-Design.ipynb), where several [RunSingleCase](#RunSingleCase) drivers are declared as direct sub-drivers of the solver.
By default, a `NonLinearSolver` driver comes with one [RunSingleCase](#RunSingleCase) child, called *runner*. 


![scipysolver](images/drivers_nonlinear.svg)


In [ ]:
from cosapp.drivers import NonLinearSolver

m = MultiplyWithResidue('mult')
solver = m.add_driver(NonLinearSolver('solver', verbose=True))
solver.runner.offdesign.add_unknown('p_in.x')
m.run_drivers()

print(m.drivers)
print(m.drivers['solver'].children)
print(m.p_in.x, m.K1, m.expected_output, m.p_out.x, m.residues.values())

### RunSingleCase

`RunSingleCase` executes all subsystem drivers by recursively calling the `compute()` method of the top system and each of its children.

This `Driver` does not contain a solver per se, but is helpfull to set boundary conditions, initial values, and define additional unknowns and/or equations (see [Advanced Drivers](./03b-Advanced-Drivers.ipynb) tutorial).
It is primarily meant to be used as a child of a [NonLinearSolver](#NonLinearSolver) driver.

To update its `System` owner, two possibilities are offered:

- *value modification* through `set_values` method will change an input value
- *initial value definition* through `set_init` method will change the value of iteratives before resolving the case.

In [ ]:
from cosapp.drivers import RunSingleCase

m = MultiplyWithResidue('mult')
update = m.add_driver(RunSingleCase('update', verbose=True))
update.offdesign.add_unknown('p_in.x')

update.set_values({'expected_output': 15.})
update.set_init({'K1': 2.})
m.run_drivers()

print('List of defined drivers\n', m.drivers)
print('\np_in.x\tK1\tp_out.x\tresidue')
print(m.p_in.x, m.K1, m.p_out.x, m.residues.values(), sep="\t")

![updatesystem](images/drivers_4.svg)

In [ ]:
m.drivers.clear() # Remove all drivers on the system `m`
solver = m.add_driver(NonLinearSolver('solver', verbose=True))
update = solver.add_child(RunSingleCase('update', verbose=True))
update.offdesign.add_unknown('p_in.x')

# Customization of the case
update.set_values({'expected_output': 15.})

# Execution
m.run_drivers()

print(m.drivers)
print(m.p_in.x, m.K1, m.expected_output, m.p_out.x, m.residues.values())

**Congrats!** You are now ready to launch computation on your `System` with **CoSApp**!

But we recommend you to read the [Advanced Drivers](./03b-Advanced-Drivers.ipynb) tutorial, before 
creating your test case.